
From the book (chapter 14.3):

>The United States Social Security Administration (SSA) has made available data on the frequency of baby names from 1880 through the present. [...] As of this writing, the US Social Security Administration makes available data files, one per year, containing the total number of births for each sex/name combination. The raw archive of these files can be obtained from [http://www.ssa.gov/oact/baby names/limits.html](http://www.ssa.gov/oact/baby names/limits.html).

___



In [ ]:
library(tidyverse)
library(magrittr)


___

## Loading the data

This dataset is split by year, so we'll assemble these files into a single dataframe, to which we'll add a new `year` column.



In [ ]:
years <- c(1880:2010)
names <- tibble("name" = NA, "sex" = NA, "births" = NA, "year" = NA)

for (year in years) {
    path <- str_c("../datasets/babynames/yob", year, ".txt", sep = "")
    frame <- read_csv(path, col_names = c("name", "sex", "births"),
                      col_types = cols(col_character(), col_character(), col_integer()))
    frame %<>% mutate("year" = year)
    names %<>% bind_rows(frame)
}
names %<>% drop_na()

In [ ]:
names


We can now start aggregating this data by year and sex.



In [ ]:
total_births <- names %>%
    group_by(year, sex) %>%
    summarise(tot_births = sum(births))

In [ ]:
total_births



Let's plot these data to view the total number of births per sex from 1880.



In [ ]:
total_births %>% ggplot(aes(x = year, y = tot_births, color = sex)) +
    geom_line() +
    labs(x = "Year", y = "Births",
         title = "Total number of births per sex") +
    theme(legend.position = c(0.1, 0.86),
          legend.background = element_rect(fill = "transparent"))


Now we'll insert a new column that will contain the proportion of babies with each name relative to the total number of births for a given year.
I had a few issues with finding a good way to apply some sort of mapping to the data, as done in the original example in Python, but I'm still quite scratching the surface of the `tidyverse`... I managed to perform a join between the `names` table and one with the total number of births per year and sex, and then calculate the desired ratio from that.



In [ ]:
births_year_sex <- names %>%
    group_by(year, sex) %>%
    summarise(tot_births = sum(births))

names %<>% inner_join(births_year_sex, by = c("year", "sex"))

In [ ]:
names %<>%
    mutate(prop = births / tot_births) %>%
    select(-tot_births)

In [ ]:
names



Let's subset these data to the top 1000 names for each year/sex combination.



In [ ]:
top1000 <- names %>%
    group_by(year, sex) %>%
    top_n(1000, wt = births)

In [ ]:
top1000



## Analyzing naming trends

Let's first split the `top1000` dataset into boys and girls subsets.



In [ ]:
boys <- top1000 %>%
    filter(sex == "M")
girls <- top1000 %>%
    filter(sex == "F")


We can now plot some data about occurrence of specific names during time.



In [ ]:
top1000 %>%
    filter((name == "John" & sex == "M") | (name == "Harry" & sex == "M") |
           (name == "Mary" & sex == "F") | (name == "Marilyn" & sex == "F")) %>%
    ggplot(aes(x = year, y = births, color = name)) +
    geom_line() +
    facet_grid(name ~ ., scales = "free_y") +
    labs(x = "Year", y = "Births",
         title = "Number of births per year",
         subtitle = "John, Harry, Mary, Marilyn") +
    scale_x_continuous(breaks = c(1880, 1900, 1920, 1940, 1960, 1980, 2000)) +
    guides(color = FALSE)

In [ ]:
# (Another useful plot for the same data might be this one. I actually prefer it better.)
top1000 %>%
    filter((name == "John" & sex == "M") | (name == "Harry" & sex == "M") |
           (name == "Mary" & sex == "F") | (name == "Marilyn" & sex == "F")) %>%
    ggplot(aes(x = year, y = births, color = name)) +
    geom_line() +
    labs(x = "Year", y = "Births",
         title = "Number of births per year",
         subtitle = "John, Harry, Mary, Marilyn") +
    scale_x_continuous(breaks = c(1880, 1900, 1920, 1940, 1960, 1980, 2000)) +
    theme(legend.position = c(0.92, 0.8),
          legend.background = element_rect(fill = "transparent"))


### Measuring the increase in naming diversity

A possible explanation for the dicrease shown in the previous plots is that fewer parents are choosing common names like Harry, John, Mary and Marilyn for their children, during latest years. Let's confirm this hypothesis with actual data.

Let's first plot the proportion of the top 1000 names to see their trend.



In [ ]:
top1000 %>%
    group_by(year, sex) %>%
    summarise(tot_prop = sum(prop)) %>%
    ggplot(aes(x = year, y = tot_prop, color = sex)) +
    geom_line() +
    scale_y_continuous(limits = c(0.0, 1.0)) +
    scale_x_continuous(breaks = c(1880, 1900, 1920, 1940, 1960, 1980, 2000)) +
    labs(x = "Year", y = "",
         title = "Sum of proportion of top 1000 common names") +
    theme(legend.position = c(0.94, 0.16),
          legend.background = element_rect(fill = "transparent"))


This plot indeed shows a decrease in using common names roughly starting from the half of the 20^th^ century.
Let's now take the number of distinct names, sorted by popularity, in the top 50% of births. We'll start with boys born in 2010.



In [ ]:
df <- boys %>%
    filter(year == 2010)
df

In [ ]:
df %<>%
    arrange(desc(prop)) %>%
    mutate(cumprop = cumsum(prop))
sum(df$cumprop <= 0.5)


Now we can check the same amount for boys born in 1900.



In [ ]:
df <- boys %>%
    filter(year == 1900)
df %<>% arrange(desc(prop)) %>%
    mutate(cumprop = cumsum(prop))
sum(df$cumprop <= 0.5)


Let's now apply this operation for every year/sex combination in the `top1000` dataset.



In [ ]:
get_quantile_count <- function(group, q = 0.5) {
    group <- group %>% arrange(desc(prop)) %>%
        mutate(cumprop = cumsum(prop))

    return(sum(group$cumprop <= q))
}

In [ ]:
diversity <- top1000 %>%
    group_by(year, sex) %>%
    do(count = get_quantile_count(.)) %>%
    mutate(count = unlist(count))

In [ ]:
diversity


This new dataframe can be plotted just like before.



In [ ]:
diversity %>%
    ggplot(aes(x = year, y = count, color = sex)) +
    geom_line() +
    scale_x_continuous(breaks = c(1880, 1900, 1920, 1940, 1960, 1980, 2000)) +
    labs(x = "Year", y = "Count",
         title = "Number of popular names in top 50%") +
    theme(legend.position = c(0.1, 0.9),
          legend.background = element_rect(fill = "transparent"))


The plot shows that female names have always been much more diverse than male names, and this is even more evident in recent times.

___



In [ ]:
sessionInfo()